# Run Instrumental Variable Analysis

### Authors: Calvin Howard.

- Similar to a mediation analysis, this removes the influence of confounds from an independent variable to attempt to isolate the causal effect upon the dependent variable. 
- For further information, Causal Inference by Scott Cunningham has an excellent chapter on IV analysis. 


# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

Prep Output Direction

In [ ]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/subiculum_cognition_and_age/figures/Figures/joint_distribution_calculus/validation_cohort'

Import Data

In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/metadata/master_list_proper_subjects.xlsx'
# Specify sheet name as a string if using Excel, otherwise set to None 
sheet = 'master_list_proper_subjects'

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()


# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

Enter names of columns you'd like to drop nans from

In [ ]:
drop_list = ['Age', 'Z_Scored_Percent_Cognitive_Improvement']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
data_df

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
data_df.columns

Set the parameters for dropping rows

In [ ]:
column = 'City'  # The column you'd like to evaluate
condition = 'not'  # The condition to check ('equal', 'above', 'below', 'not')
value = 'Toronto' # The value to drop if T

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['Ordinal_Target_Type', 'Ordinal_Epilepsy_Type']

In [ ]:
data_df = cal_palm.standardize_columns(cols_not_to_standardize)
data_df

In [ ]:
data_df.columns

Regress out Covariate

In [ ]:
from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
# use this code block to regress out covariates. Generally better to just include as covariates in a model..
dependent_variable_list = ['dependent_variable_column']
regressors = ['Age', 'Sex']

data_df, adjusted_dep_vars_list = RegressOutCovariates.run(df=data_df, dependent_variable_list=dependent_variable_list, covariates_list=regressors)
print(adjusted_dep_vars_list)

# 01B - Import Directly from a CSV

In [ ]:
import pandas as pd
data_df = pd.read_csv('path/to/your/csv.csv')

# 02 - Evaluate Instrumental Variable
- There are 3 assumptions to an IV analysis. This will go over them. 

In [ ]:
import statsmodels.api as sm
import patsy
from scipy.stats import pearsonr

def relevance_assumption(data_df, iv, second_var):
    formula = f'{second_var} ~ {iv}'
    y, X = patsy.dmatrices(formula, data=data_df, return_type='dataframe')
    first_stage = sm.OLS(y, X).fit()
    f_stat = first_stage.fvalue
    print(f"Relevance Assumption F-Statistic: {f_stat}")
    if f_stat > 10:
        print("The instrument is acceptable (F-statistic > 10).")
    else:
        print("The instrument is weak (F-statistic ≤ 10). Suggest not using this instrument")
    return f_stat

def exogeneity_assumption(data_df, iv, second_var, dependent_var):
    formula = f'{dependent_var} ~ {second_var}'
    y, X = patsy.dmatrices(formula, data=data_df, return_type='dataframe')
    ols_reg = sm.OLS(y, X).fit()
    residuals = ols_reg.resid
    r_value, p_value = pearsonr(data_df[iv], residuals)
    print(f"Exogeneity Assumption: Pearson correlation between residuals and IV: R = {r_value}, P = {p_value}")
    if p_value > 0.05:
        print("Exogeneity assumption has been met (P > 0.05).")
    else:
        print("Exogeneity assumption failed (P ≤ 0.05).")
    return r_value, p_value

def exclusion_assumption():
    print("Reminder: The exclusion assumption states that the instrumental variable should not be influenced by confounders.")
    print("This assumption must be proven through theory and domain knowledge, not through statistical tests.")
    
def check_iv_assumptions(data_df, iv, second_var, dependent_var):
    print("Checking Relevance Assumption:")
    relevance_assumption(data_df, iv, second_var)
    
    print("\nChecking Exogeneity Assumption:")
    exogeneity_assumption(data_df, iv, second_var, dependent_var)
    
    print("\nChecking Exclusion Assumption:")
    exclusion_assumption()

In [ ]:
data_df.columns

In [ ]:
instrumental_variable = 'Age'
independent_variable = 'Subiculum_Grey_Matter'
dependent_var = 'Standardized_Percent_Improvement'

In [ ]:
check_iv_assumptions(data_df, iv=instrumental_variable, second_var=independent_variable, dependent_var=dependent_var)

# 03 - Run IV analysis

In [ ]:
import pandas as pd
import statsmodels.api as sm
import patsy

def iv_analysis(data_df, iv, second_var, dependent_var):
    # 1. OLS regression using patsy formula
    ols_formula = f'{dependent_var} ~ {second_var}'
    y_ols, X_ols = patsy.dmatrices(ols_formula, data=data_df, return_type='dataframe')
    ols_model = sm.OLS(y_ols, X_ols).fit()
    
    # 2. First stage regression (IV as the predictor for second_var)
    first_stage_formula = f'{second_var} ~ {iv}'
    y_first_stage, X_first_stage = patsy.dmatrices(first_stage_formula, data=data_df, return_type='dataframe')
    first_stage_model = sm.OLS(y_first_stage, X_first_stage).fit()
    data_df[second_var] = first_stage_model.resid
    
    # 3. IV (2SLS) regression using predicted values from first stage
    iv_formula = f'{dependent_var} ~ {second_var}'
    y_iv, X_iv = patsy.dmatrices(iv_formula, data=data_df, return_type='dataframe')
    iv_model = sm.OLS(y_iv, X_iv).fit()
    
    # 4. Create DataFrame for the results
    results_df = pd.DataFrame({
        'Coefficient (IV)': iv_model.params,
        'Standard Error (IV)': iv_model.bse,
        'P-Value (IV)': iv_model.pvalues,
        'Coefficient (OLS)': ols_model.params,
        'Standard Error (OLS)': ols_model.bse,
        'P-Value (OLS)': ols_model.pvalues
    })
    
    # Formatting the coefficients and standard errors
    results_df['Coefficient (IV)'] = results_df.apply(
        lambda row: f"{row['Coefficient (IV)']:.4f} ({row['Standard Error (IV)']:.4f})", axis=1
    )
    results_df['Coefficient (OLS)'] = results_df.apply(
        lambda row: f"{row['Coefficient (OLS)']:.4f} ({row['Standard Error (OLS)']:.4f})", axis=1
    )
    
    # Reordering the columns for readability
    results_df = results_df[['Coefficient (IV)', 'P-Value (IV)', 'Coefficient (OLS)', 'P-Value (OLS)']]
    
    # Explanation of the comparison
    print("--Comparison between OLS and 2SLS coefficients--")
    print("If the OLS is unbiased, the coefficient from the IV analysis should be different from the OLS analysis. This coefficient is the 'causal effect' of the Dep var on the Indep var")
    return results_df



In [ ]:
iv_analysis(data_df, iv=instrumental_variable, second_var=independent_variable, dependent_var=dependent_var)